# Using the NY Times API and Twilio (using Python 3.x)

 - In this brief tutorial, we will go through how to get a list of the current best sellers from the New York Times. Using their API, we will parse through the JSON, and looking at the data in a nicely formatted table string.
 - Once we have worked through the process of using the NY Times API, we will go through how we can use that data to send a text message with the information to any phone number(s) using Twilio.
 
<br>
Twilio is an important tool currently for marketing campaigns. It can be used to send programmable text messages and voice calls. I suspect many have received solicited phone call from marketers using Twilio, here we will explore its use!

***

## Using New York Times Best Sellers API
 - Set up account <a href='https://developer.nytimes.com/signup'>here</a> to get API key, make sure to choose Books API in the drop down; the website does not matter.

In [1]:
import requests

In [2]:
BOOKS_API = 'YOUR_API_KEY'
url = 'https://api.nytimes.com/svc/books/v3/lists.json?api-key=%s&list=%s'
sections = ['hardcover-nonfiction']# 'combined-print-and-e-book-nonfiction' 

In [3]:
books_url = url % (BOOKS_API, sections[0])

In [4]:
rsp = requests.get(books_url)
assert rsp.status_code == 200, "Ruh Roh! code: %d"%rsp.status_code

Passed that assertion test (meaning we got an OK response 200), let's take a look at what we got back from the request. We requested json in our URL so we can take advantage of requests library's json method.

In [5]:
rsp.json()

{'copyright': 'Copyright (c) 2017 The New York Times Company.  All Rights Reserved.',
 'last_modified': '2017-07-28T13:38:01-04:00',
 'num_results': 15,
 'results': [{'amazon_product_url': 'https://www.amazon.com/Devils-Bargain-Bannon-Storming-Presidency/dp/0735225028?tag=NYTBS-20',
   'asterisk': 0,
   'bestsellers_date': '2017-07-22',
   'book_details': [{'age_group': '',
     'author': 'Joshua Green',
     'contributor': 'by Joshua Green',
     'contributor_note': '',
     'description': 'A deeply reported account of the relationship between Donald Trump and his chief strategist.',
     'price': 0,
     'primary_isbn10': '0735225028',
     'primary_isbn13': '9780735225022',
     'publisher': 'Penguin Press',
     'title': "DEVIL'S BARGAIN"}],
   'dagger': 0,
   'display_name': 'Hardcover Nonfiction',
   'isbns': [{'isbn10': '0735225028', 'isbn13': '9780735225022'},
    {'isbn10': '0735225036', 'isbn13': '9780735225039'}],
   'list_name': 'Hardcover Nonfiction',
   'published_date': 

***
We get a whole bunch of stuff back in the request, let's focus on the results for the first book. (Note that this information is attributed to New York Times)

In [6]:
rsp.json()['results'][0]

{'amazon_product_url': 'https://www.amazon.com/Devils-Bargain-Bannon-Storming-Presidency/dp/0735225028?tag=NYTBS-20',
 'asterisk': 0,
 'bestsellers_date': '2017-07-22',
 'book_details': [{'age_group': '',
   'author': 'Joshua Green',
   'contributor': 'by Joshua Green',
   'contributor_note': '',
   'description': 'A deeply reported account of the relationship between Donald Trump and his chief strategist.',
   'price': 0,
   'primary_isbn10': '0735225028',
   'primary_isbn13': '9780735225022',
   'publisher': 'Penguin Press',
   'title': "DEVIL'S BARGAIN"}],
 'dagger': 0,
 'display_name': 'Hardcover Nonfiction',
 'isbns': [{'isbn10': '0735225028', 'isbn13': '9780735225022'},
  {'isbn10': '0735225036', 'isbn13': '9780735225039'}],
 'list_name': 'Hardcover Nonfiction',
 'published_date': '2017-08-06',
 'rank': 1,
 'rank_last_week': 0,
 'reviews': [{'article_chapter_link': '',
   'book_review_link': '',
   'first_chapter_link': '',
   'sunday_review_link': ''}],
 'weeks_on_list': 1}

Nice! This gives us a feel of what each entry of the result will look like, for this tutorial we will be focusing on the book details, just providing the list of what books are current best sellers, as well as how long they have appeared on the list. Let's store the JSON response in a variable to work with.

In [7]:
json_resp = rsp.json()['results']

Now we can simply loop through this to get each book title, author and number of weeks it has been on the list. Formatting our results into a table like structure.

In [8]:
print('+'+'-'*102+'+')
print('|{:^40}|{:^40}|{:^20}|'.format('Title', 'Author(s)', 'Weeks On List'))
print('|'+'-'*40+ '+'+'-'*40+'+'+'-'*20+'|')

for book in json_resp:
    deets = book['book_details'][0]
    print('|{:^40}|{:^40}|{:20}|'.format(deets['title'], deets['author'], book['weeks_on_list']))

print('+'+'-'*102+'+')

+------------------------------------------------------------------------------------------------------+
|                 Title                  |               Author(s)                |   Weeks On List    |
|----------------------------------------+----------------------------------------+--------------------|
|            DEVIL'S BARGAIN             |              Joshua Green              |                   1|
|   ASTROPHYSICS FOR PEOPLE IN A HURRY   |          Neil deGrasse Tyson           |                  12|
|            HILLBILLY ELEGY             |               J D Vance                |                  52|
|    AL FRANKEN, GIANT OF THE SENATE     |               Al Franken               |                   8|
|               DANGEROUS                |           Milo Yiannopoulos            |                   3|
|       REDISCOVERING AMERICANISM        |              Mark R Levin              |                   4|
|          UNDERSTANDING TRUMP           |             

There we go! You now have a basic understanding of how to use the New York Times Best Sellers API. Now let's try to tie this into sending a text with this information using Twilio!
***

### Using Twilio
 - First you will need to set up a <a href='https://www.twilio.com/try-twilio'>free account</a> to obtain our API keys
 - Next make sure you have the twilio package installed (uncomment first line of next cell to install)

In [9]:
# !pip install twilio
from twilio.rest import Client

Now that we have everything installed and imported, and the API keys, let's put everything together!

In [10]:
text_body = 'NY Times non-fiction best sellers: \n'
for rank,book in enumerate(json_resp):
    details = book['book_details'][0]
    text_body += '\n{}. {} by {} ({})\n'.format(rank+1, 
                                                details['title'],
                                                details['author'],
                                                book['weeks_on_list'])

In [11]:
print(text_body)

NY Times non-fiction best sellers: 

1. DEVIL'S BARGAIN by Joshua Green (1)

2. ASTROPHYSICS FOR PEOPLE IN A HURRY by Neil deGrasse Tyson (12)

3. HILLBILLY ELEGY by J D Vance (52)

4. AL FRANKEN, GIANT OF THE SENATE by Al Franken (8)

5. DANGEROUS by Milo Yiannopoulos (3)

6. REDISCOVERING AMERICANISM by Mark R Levin (4)

7. UNDERSTANDING TRUMP by Newt Gingrich (6)

8. OPTION B by Sheryl Sandberg and Adam Grant (13)

9. KILLERS OF THE FLOWER MOON by David Grann (14)

10. YOU DON'T HAVE TO SAY YOU LOVE ME by Sherman Alexie (5)

11. THE SWAMP by Eric Bolling (4)

12. I CAN'T MAKE THIS UP by Kevin Hart with Neil Strauss (7)

13. THE OPERATOR by Robert O'Neill (7)

14. BORN A CRIME by Trevor Noah (22)

15. HUE 1968 by Mark Bowden (7)



The formatting here is subjective to the phone you want to send to, this format is decent on an iPhone, adjust accordingly.
***
That gives us the body of the text to send, all that is left is setting up the Twilio client; onward!

In [12]:
# Your Account SID and AUTH Token from twilio.com/console
client = Client(YOUR_TWILIO_ACCOUNT_SID, YOUR_TWILIO_AUTH_TOKEN)
message = client.messages.create(to = VALID_PHONE_NUMBER,
                                 from_ = YOUR_TWILIO_NUMBER,
                                 body = text_body)

## Finished
That is all for this one, hopfully this is useful. I have this program running as a cron job (weekly task) to send the list to a set people using my blog site. Could easily wrap this in a while loop, checking time, sending the list once a week as well.